# Example Notebook demonstarting some basic functionality

### Working with legacy StatSTEMInput data

In [ ]:
import matplotlib.pyplot as plt

import qem

legacyStatSTEM = qem.io.read_legacyInputStatSTEM('../data/Au/Example_Au.mat')
image = legacyStatSTEM['input']['obs']
dx = legacyStatSTEM['input']['dx']
inputStatSTEM = legacyStatSTEM['input']
outputStatSTEM = legacyStatSTEM['output']

input_coordinates = inputStatSTEM['coordinates']
output_coordinates = outputStatSTEM['coordinates']
input_coordinates[:,1] = input_coordinates[:,1]/dx
input_coordinates[:,0] = input_coordinates[:,0]/dx
from qem.image_fitting import ImageFitting

model=ImageFitting(image, dx=dx, elements=['Au'])
model.import_coordinates(input_coordinates -1)
model.plot()

In [ ]:
model.fit_background = True
model.same_width =True
model.model_type = 'gaussian'
params = model.init_params()
params = model.linear_estimator(params)
model.prediction = model.predict(local=True)
# model.fit_global(params, maxiter=1000, tol = 1e-6, verbose=True)
model.fit_stochastic(params, maxiter=50, tol =1e-3,batch_size=3000, num_epoch=5, verbose=1, step_size=0.1, plot=False)
# model.fit_voronoi(params=params)


In [ ]:
model.fit_voronoi()


In [ ]:
model.plot_scs()

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(2,3,1)
im=plt.imshow(model.image)
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.tight_layout()
plt.title('Input Image')

plt.subplot(2,3,2)
im=plt.imshow(model.prediction)
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.clim(model.image.min(),model.image.max())
plt.tight_layout()
plt.title('QEM Model')

plt.subplot(2,3,3)
diff = model.image - model.prediction
im=plt.imshow(diff)
plt.colorbar(im,fraction=0.046, pad=0.04)
# plt.clim([-1000,1000])
plt.tight_layout()
plt.title('Residuals')

plt.subplot(2,3,4)
im=plt.imshow(image)
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.tight_layout()
plt.title('Input Image')

plt.subplot(2,3,5)
im=plt.imshow(outputStatSTEM['model'])
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.clim(model.image.min(),model.image.max())   
plt.tight_layout()
plt.title('Legacy StatSTEM Model')

plt.subplot(2,3,6)
im=plt.imshow(inputStatSTEM['obs']-outputStatSTEM['model'])
plt.colorbar(im,fraction=0.046, pad=0.04)
# plt.clim([-1000,1000])
plt.tight_layout()
plt.title('Residuals')

In [ ]:
import numpy as np
from qem.utils import safe_convert_to_numpy
res = model.residual(model.params)
our = np.std(res)
ref = np.std(inputStatSTEM['obs']-outputStatSTEM['model'])

print("QEM residual std: ", our)
print("Legacy StatSTEM residual std: ", ref)
